## Load video and gloss data, and prepare for classification

1 load the segmented gloss videos

2 extract labels for data; get data statistics

3 split videos into gloss chunks

3.1 turn those gloss chunks into frames and save frames as array

4 create sentences dataframe and dataframe of frame arrays

In [19]:
#1
#!pip install ffmpeg
#!pip install Pillow

from PIL import Image
import os
import shutil
import pandas as pd
import ffmpeg
import numpy as np
from numpy import asarray
import cv2 as cv
import re
import math
from subprocess import check_call, PIPE, Popen
import shlex
from tqdm import tqdm



---



## Get statistics for ECHO and CorpusNGT:

First we have to split the CorpusNGT videos by speaker

In [4]:
def cleanItem(item):
    if "(p-)" in item:  item = item.replace("(p-)", "")
    if "(g-)" in item:  item = item.replace("(g-)", "")
    if "/" in item:  item = item.split("/")[0] # make sure the gloss name is regularized:
    if " (" in item:  item = item.split(" (")[0]
    if ") " in item:  item = item.split(") ")[1]
    if ")-" in item:  item = item.split(")-")[1]
    if " " in item:  item = item.replace(" ", "")
    if '"' in item:  item = item.replace('"', "")
    if "'" in item:  item = item.replace("'", "")
    if "~" in item:  item = item.replace("~", "")
    if "." in item:  item = item.replace(".", "-")
    if "_" in item:  item = item.replace("_", "-")
    if "?" in item:  item = item.replace("?", "")
    if ">" in item:  item = item.replace(">", "")
    if "<" in item:  item = item.replace("<", "")
    if "%" in item:  item = item.replace("%", "")
    if "#" in item:  item = item.replace("#", "")
    if "!" in item:  item = item.replace("!", "")
    if "$" in item:  item = item.replace("$", "")
    if "*" in item:  item = item.replace("*", "")
    if "&" in item:  item = item.replace("&", "")
    if "+" in item:  item = item.replace("+", "")
    if "`" in item:  item = item.replace("`", "")
    if "-1" in item:  item = item.replace("-1", "")
    if "-2" in item:  item = item.replace("-2", "")
    if "-3" in item:  item = item.replace("-3", "")
    if "-4" in item:  item = item.replace("-4", "")
    if "-5" in item:  item = item.replace("-5", "")
    if "-6" in item:  item = item.replace("-6", "")
    if "-7" in item:  item = item.replace("-7", "")
    if "-8" in item:  item = item.replace("-8", "")
    if "-9" in item:  item = item.replace("-9", "")
    if "-0" in item:  item = item.replace("-0", "")
    if "-EC" in item:  item = item.split("-EC")[0]
    if "-HC" in item:  item = item.split("-HC")[0]
    if "+PT" in item:  item = item.replace("+PT", "")
    if "+PO" in item:  item = item.replace("+PO", "")
    if "-RC" in item:  item = item.replace("-RC", "")
    if "-A" in item:  item = item.replace("-A", "")
    if "-B" in item:  item = item.replace("-B", "")
    if "-C" in item:  item = item.replace("-C", "")
    if "-D" in item:  item = item.replace("-D", "")
    if "-E" in item:  item = item.replace("-E", "")
    if "-F" in item:  item = item.replace("-F", "")
    if "-G" in item:  item = item.replace("-G", "")
    if "-H" in item:  item = item.replace("-H", "")
    if "(H)" in item:  item = item.replace("(H)", "")
    if "(-H)" in item:  item = item.replace("(-H)", "")
    if "(2H)" in item:  item = item.replace("(2H)", "")
    item = item.upper()
    return(item)

In [5]:
ngtglossdir = "Data/CorpusNGT/gloss/"

for filename in os.listdir(ngtglossdir):
    f = os.path.join(ngtglossdir, filename)
    file, extension = filename.split('.')
    # For the csv file, we get all the statistics like gloss count etc
    if extension == 'mpg':
        os.rename('{}{}.mpg'.format(ngtglossdir, file), '{}{}.mpg'.format(ngtglossdir, file.replace('_b', "")))

keepvideofiles = []

for filename in os.listdir(ngtglossdir):
    f = os.path.join(ngtglossdir, filename)
    file, extension = filename.split('.')
    # For the csv file, we get all the statistics like gloss count etc

    if extension == 'csv':
        if "_S" not in file:
            gloss = pd.read_csv(f, header=None, names = ['signer', 'start time', 'start seconds', 'stop time', 'stop seconds', 'duration', 'value', 'gloss'])
            gloss = gloss.dropna(axis=0)

            for item in gloss['gloss']:
                item = cleanItem(item)

            gloss = gloss.dropna(axis=0)
            signers = gloss['signer'].unique()
            gloss = gloss.set_index('signer', drop=True, append=False)

            if len(signers) == 0:
                print("No speakers in file, check absence of speakers for file " + file)
            elif len(signers) == 1:
                print("Only one speaker for file  " + file)
                S1_number = signers[0]
                try:
                    gloss_S1 = gloss.loc[[S1_number]]
                    gloss_S1.to_csv('{}{}_{}.csv'.format(ngtglossdir, file, S1_number))
                    keepvideofiles.append('{}{}_{}.mpg'.format(ngtglossdir, file, S1_number))
                except:
                    print("Error with file: " + file + ", Check manually")
                    #try:  os.remove('{}{}_{}.mpg'.format(ngtglossdir, file, S1_number))
                    #except: pass
            elif len(signers) == 2:
                S1_number = signers[0]
                S2_number = signers[1]
                try:
                    gloss_S1 = gloss.loc[[S1_number]]
                    gloss_S1.to_csv('{}{}_{}.csv'.format(ngtglossdir, file, S1_number))
                    keepvideofiles.append('{}{}_{}.mpg'.format(ngtglossdir, file, S1_number))
                    gloss_S2 = gloss.loc[[S2_number]]
                    gloss_S2.to_csv('{}{}_{}.csv'.format(ngtglossdir, file, S2_number))
                    keepvideofiles.append('{}{}_{}.mpg'.format(ngtglossdir, file, S2_number))
                except:
                    print("Error with file: " + file + ", Check manually")
                    #try:  os.remove('{}{}_{}.mpg'.format(ngtglossdir, file, S2_number))
                    #except: pass

            os.remove('{}{}.csv'.format(ngtglossdir, file))

Only one speaker for file  CNGT0047
Only one speaker for file  CNGT0048
Only one speaker for file  CNGT0092
Only one speaker for file  CNGT0211
Only one speaker for file  CNGT0315
Only one speaker for file  CNGT0318
Only one speaker for file  CNGT0323
No speakers in file, check absence of speakers for file CNGT0324
Only one speaker for file  CNGT0325
Only one speaker for file  CNGT0327
Only one speaker for file  CNGT0413
Only one speaker for file  CNGT0417
Only one speaker for file  CNGT0423
Only one speaker for file  CNGT0469
Only one speaker for file  CNGT0470
Only one speaker for file  CNGT0471
Only one speaker for file  CNGT0473
Only one speaker for file  CNGT0474
Only one speaker for file  CNGT0475
Only one speaker for file  CNGT0602
Only one speaker for file  CNGT0603
Only one speaker for file  CNGT0604
Only one speaker for file  CNGT0605
Only one speaker for file  CNGT0606
Only one speaker for file  CNGT0607
Only one speaker for file  CNGT0704
Only one speaker for file  CNGT0708

In [6]:
for filename in os.listdir(ngtglossdir):
    f = os.path.join(ngtglossdir, filename)
    if f.split(".")[1] == "mpg":
        if f not in keepvideofiles:
            os.remove(f)

heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo
heyo


In [9]:
!pip install moviepy

from moviepy.video.io.VideoFileClip import VideoFileClip

     ---------------------------------------- 0.0/388.3 kB ? eta -:--:--
     - -------------------------------------- 10.2/388.3 kB ? eta -:--:--
     ------------ ------------------------- 122.9/388.3 kB 1.8 MB/s eta 0:00:01
     -------------------------------------- 388.3/388.3 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/22.6 MB ? eta -:--:--
    --------------------------------------- 0.3/22.6 MB 7.0 MB/s eta 0:00:04
   - -------------------------------------- 0.6/22.6 MB 7.2 MB/s eta 0:00:04
   - -------------------------------------- 1.0/22.6 MB 7.3 MB/s eta 0:00:03
   -- ------------------------------------- 1.4/22.6 MB 7.5 MB/s eta 0:00:03
   --- ------------------------------------ 1.8/22.6 MB 7.7 MB/s eta 0:00:03
   ---- ----------------------------------- 2.4/22.6 MB 8.3 MB/s eta 0:00:03
   ---- ----------------------------------- 2.8/22.6 MB 8.8

In [11]:
total_unique_glosses = dict()

def getStats(directorygloss, directorytrans, trans=True, CorpusNGT=False):
    # Gets statistics like average video duration, number of translated videos, number of unique glosses, number of sentences, avg number of sentences etc
    # for every video in the ECHO corpus.

    unique_glosses = dict()
    gloss_count = dict()
    translated_video_count = 0
    glossed_video_count = 0
    sentence_count = 0
    video_lengths = []
    video_gloss_counts = []

    if trans:
        for filename in os.listdir(directorytrans):
            f = os.path.join(directorytrans, filename)
            file, extension = filename.split('.')

            # For the csv file, we get all the statistics like gloss count etc
            if extension == 'csv':
                translated_video_count += 1
                glossed_video_count += 1
                gloss = pd.read_csv(f, header=None, names = ['signer', 'start time', 'start seconds', 'stop time', 'stop seconds', 'value', 'gloss'])
                gloss = gloss.dropna(axis=0)

                try:
                    sentences = gloss.loc[["Translation Dutch"]]
                    sentence_count += len(sentences)
                except:
                    pass
                try:
                    sentences = gloss.loc[["Dutch Translation"]]
                    sentence_count += len(sentences)
                except:
                    pass

    for filename in os.listdir(directorygloss):
        f = os.path.join(directorygloss, filename)
        file, extension = filename.split('.')
        # For the csv file, we get all the statistics like gloss count etc

        if extension == 'csv':
            glosses_in_video = 0
            glossed_video_count += 1
            gloss = pd.read_csv(f, header=None, names = ['signer', 'start time', 'start seconds', 'stop time', 'stop seconds', 'value', 'gloss'])
            gloss = gloss.dropna(axis=0)

            if not CorpusNGT:
                gloss = gloss.loc[["Gloss RH", "Gloss LH"]]

            for item in gloss['gloss']:
                item = cleanItem(item)
                if item == '':
                    continue
                glosses_in_video += 1
                if item not in unique_glosses:
                    n = len(unique_glosses)
                    unique_glosses[item] = n
                    if item not in total_unique_glosses:
                        n = len(total_unique_glosses)
                        total_unique_glosses[item] = n
                    gloss_count[item] = 0
                gloss_count[item] = gloss_count[item]+1
            video_gloss_counts.append(glosses_in_video)

        if extension == 'mpg':
            video = VideoFileClip(f)
            duration = video.duration
            video_lengths.append(duration)
            video.reader.close()

    average_video_gloss_count = sum(video_gloss_counts)/len(video_gloss_counts)
    average_video_length = sum(video_lengths)/len(video_lengths)
    return(translated_video_count, glossed_video_count, unique_glosses, gloss_count, average_video_length, sentence_count, average_video_gloss_count)

def writeStats(translated_video_count, glossed_video_count, unique_glosses, gloss_count, average_video_length, sentence_count, average_video_gloss_count, name='data_statistics.txt', CorpusNGT=False):
    with open('Data/' + name, 'w') as f:
        f.write("Nr of glossed videos in ECHO: \t\t\t" + str(glossed_video_count) + "\n")
        f.write("Nr of unique glosses in ECHO: \t\t\t" + str(len(unique_glosses)) + "\n")
        f.write("Nr of total glosses in ECHO: \t\t\t" + str(sum(gloss_count.values())) + "\n")
        f.write("Average length of videos in ECHO: \t\t" + str(average_video_length) + " sec or " + str(average_video_length/60) + " min" + "\n")
        f.write("Avg number of glosses per video in ECHO: \t" + str(average_video_gloss_count) + "\n")
        if not CorpusNGT:
            f.write("Avg number of glosses per sentence in ECHO: \t" + str(average_video_gloss_count/(sentence_count/translated_video_count)) + "\n")
            f.write("Avg number of sentences per video in ECHO: \t" + str(sentence_count/translated_video_count) + "\n")
            f.write("Nr of sentences in ECHO: \t\t\t" + str(sentence_count) + "\n")
            f.write("Nr of translated videos in ECHO: \t\t" + str(translated_video_count) + "\n")

#glossdir = "Data/ECHO/gloss/"
glosstransdir = "Data/ECHO/gloss_trans/"

#translated_video_count, glossed_video_count, e_unique_glosses, e_gloss_count, average_video_length, sentence_count, average_video_gloss_count = getStats(glossdir, glosstransdir)
#writeStats(translated_video_count, glossed_video_count, e_unique_glosses, e_gloss_count, average_video_length, sentence_count, average_video_gloss_count, name='data_statistics_ECHO.txt')

#print("Nr of translated videos in ECHO: \t\t" + str(translated_video_count))
#print(("Nr of glossed videos in ECHO: \t\t\t" + str(glossed_video_count)))
#print("Nr of unique glosses in ECHO: \t\t\t" + str(len(e_unique_glosses)))
#print("Nr of total glosses in ECHO: \t\t\t" + str(sum(e_gloss_count.values())))
#print("Nr of sentences in ECHO: \t\t\t" + str(sentence_count))
#print("Average length of videos in ECHO: \t\t" + str(average_video_length) + " sec or " + str(average_video_length/60) + " min")
#print("Avg number of sentences per video in ECHO: \t" + str(sentence_count/translated_video_count))
#print("Avg number of glosses per video in ECHO: \t" + str(average_video_gloss_count))
#print("Avg number of glosses per sentence in ECHO: \t" + str(average_video_gloss_count/(sentence_count/translated_video_count)))
#print("Count for each gloss in ECHO: \t\t\t" + str(gloss_count))
#print()

glossdir = "Data/CorpusNGT/gloss/"

translated_video_count, glossed_video_count, c_unique_glosses, c_gloss_count, average_video_length, sentence_count, average_video_gloss_count = getStats(glossdir, glosstransdir, trans=False, CorpusNGT=True)
writeStats(translated_video_count, glossed_video_count, c_unique_glosses, c_gloss_count, average_video_length, sentence_count, average_video_gloss_count, name='data_statistics_CorpusNGT.txt', CorpusNGT=True)

print(("Nr of glossed videos in CNGT: \t\t\t" + str(glossed_video_count)))
print("Nr of unique glosses in CNGT: \t\t\t" + str(len(c_unique_glosses)))
print("Nr of total glosses in CNGT: \t\t\t" + str(sum(c_gloss_count.values())))
print("Average length of videos in CNGT: \t\t" + str(average_video_length) + " sec or " + str(average_video_length/60) + " min")
print("Avg number of glosses per video in CNGT: \t" + str(average_video_gloss_count))
#print("Avg number of glosses per sentence in CNGT: \t" + str(average_video_gloss_count/(sentence_count/translated_video_count)))
#print("Count for each gloss in CNGT: \t\t\t" + str(gloss_count))

Nr of glossed videos in CNGT: 			605
Nr of unique glosses in CNGT: 			2328
Nr of total glosses in CNGT: 			118420
Average length of videos in CNGT: 		157.27872516556292 sec or 2.6213120860927153 min
Avg number of glosses per video in CNGT: 	195.73553719008265


In [12]:
# Save dict of unique glosses and labels
np.save('Data/CorpusNGT/gloss_labels.npy', c_unique_glosses)
#np.save('drive/My Drive/Thesis/Data/ECHO/gloss_labels.npy', e_unique_glosses)

## Preprocess all ECHO data and CorpusNGT

Meaning, we split the videos into gloss sized chunks, and we create a csv file with all the glosses matched to sentences

In [13]:
c_unique_labels = np.load('Data/CorpusNGT/gloss_labels.npy', allow_pickle='TRUE').item()
#print(read_dictionary['WANDELEN']) # displays "world"

In [ ]:
print(c_unique_glosses)

{'GLOSS': 0, 'ZELFDE': 1, 'REGEN': 2, 'MOOI': 3, 'ALLES': 4, 'ORGANISEREN': 5, 'OPRICHTEN': 6, 'ENZOVOORTS': 7, 'AL': 8, 'BEWAREN': 9, 'RESTAURANT': 10, 'RUIMTE': 11, 'WEG': 12, 'PT': 13, 'ERG': 14, 'HOPEN': 15, 'VREEMD': 16, 'KLOPT': 17, 'VORIGE': 18, 'PIJN': 19, 'DENKEN': 20, 'WANDELEN': 21, 'HEET': 22, 'MAAR': 23, 'DAG': 24, '2': 25, '2-ORD': 26, '3-ORD': 27, 'TOCH': 28, 'PO': 29, 'DRINKEN': 30, 'ETEN': 31, 'GELD': 32, 'VERLIEZEN': 33, 'MEE': 34, 'SLECHT': 35, 'WEER': 36, 'NIEUW': 37, 'MOVE': 38, 'ZON': 39, 'DOOR': 40, 'POPT': 41, 'OPGEVEN': 42, 'LICHAAM': 43, 'BRAND': 44, 'MOE': 45, 'SOLDAAT': 46, 'RUGZAK': 47, 'MENS': 48, 'OVERLEDEN': 49, '3': 50, 'NAAST': 51, '10': 52, 'OVERGEVEN': 53, 'REANIMEREN': 54, 'R': 55, 'KLAAR': 56, 'DAN': 57, 'BETER': 58, 'HERFST': 59, '4': 60, 'TOT': 61, 'JAAR': 62, 'VROEGER': 63, 'NOOIT': 64, 'KLAGEN': 65, '5': 66, 'GEBEUREN': 67, 'VOOR': 68, 'GELOVEN': 69, 'BLIJKEN': 70, 'WACHTEN': 71, 'LAAT-MAAR': 72, 'NU': 73, 'BOVEN': 74, '30': 75, 'ONGEVEER': 76,

In [ ]:
import collections
od = collections.OrderedDict(sorted(c_unique_glosses.items()))

In [ ]:
print(od)

OrderedDict([('0', 393), ('1', 132), ('10', 52), ('100', 559), ('1000', 1214), ('11', 667), ('12', 1615), ('13', 668), ('14', 1345), ('15', 465), ('158', 2156), ('16', 480), ('17', 1501), ('18', 831), ('19', 1482), ('2', 25), ('2-ORD', 26), ('20', 580), ('22', 915), ('26', 1589), ('3', 50), ('3-ORD', 27), ('30', 75), ('4', 60), ('4-ORD', 1664), ('40', 1483), ('5', 66), ('50', 573), ('500', 1246), ('500^', 1987), ('6', 77), ('60', 954), ('7', 1315), ('70', 953), ('7UP', 1947), ('8', 78), ('80', 1573), ('9', 79), ('90', 1097), ('A', 352), ('AAL', 1968), ('AAN', 1447), ('AANBELLEN', 987), ('AANBIEDEN', 742), ('AANDACHT', 510), ('AANHALINGSTEKENS', 650), ('AANKLEDEN', 971), ('AANKLIKKEN', 646), ('AANKLOPPEN', 1966), ('AANLEUNEN', 925), ('AANMOEDIGEN', 2276), ('AANPAKKEN', 1088), ('AANPASSEN', 310), ('AANRANDEN', 1598), ('AANSLUITEN', 551), ('AANVALLEN', 1708), ('AANVALLEN-TEAMS', 1700), ('AANWEZIG', 101), ('AAP', 644), ('AAPACHTIGEBEWEGINGENMAKEN', 1727), ('AAPNADOEN', 1726), ('AARDIG', 15

In [ ]:
# first we create a folder for the gloss videos
# for each unique gloss we make a folder in our videos directory, and in our images directory
#viddir_name = "drive/My Drive/Thesis/Data/ECHO/gloss_split"
#for i in e_unique_glosses:
#    os.mkdir(os.path.join(viddir_name , str(i)))

In [14]:
# first we create a folder for the gloss videos
# for each unique gloss we make a folder in our videos directory, and in our images directory
viddir_name = "Data/CorpusNGT/gloss_split"
for i in c_unique_glosses:
    try:
        os.mkdir(os.path.join(viddir_name , str(i)))
    except:
        print("folder already exists")

folder already exists
folder already exists


In [15]:
## Define all functions again with alterations

re_metadata = re.compile('Duration: (\d{2}):(\d{2}):(\d{2})\.\d+,.*\n.* (\d+(\.\d+)?) fps')

def get_metadata(filename):
    '''
    Get video metadata using ffmpeg
    '''
    p1 = Popen(["ffmpeg", "-hide_banner", "-i", filename], stderr=PIPE, universal_newlines=True)
    output = p1.communicate()[1]
    print(output)
    matches = re_metadata.search(output)
    if matches:
        video_length = int(matches.group(1)) * 3600 + int(matches.group(2)) * 60 + int(matches.group(3))
        video_fps = float(matches.group(4))
        # print('video_length = {}\nvideo_fps = {}'.format(video_length, video_fps))
    else:
        raise Exception("Can't parse required metadata")
    return video_length, video_fps

def split_cut(path, outputpath, filename, ext, starts, stops, glosses, gloss_counter):
    '''
    Split video by cutting and re-encoding: accurate but very slow
    Adding "-c copy" speed up the process but causes imprecise chunk durations
    Reference: https://stackoverflow.com/a/28884437/1862500
    '''
    split_count = len(starts)
    input = '{}{}.{}'.format(path, filename, ext)

    # parse meta data
    video_length, video_fps = get_metadata(input)

    # split the video by gloss
    output = []

    for i in range(split_count):
        split_start = float(starts[i])
        split_size = float(stops[i]) - float(starts[i])
        counter = gloss_counter[glosses[i]]
        output_path = '{}{}/{}.{}'.format(outputpath, glosses[i], counter, ext)
        gloss_counter[glosses[i]] = gloss_counter[glosses[i]]+1
        cmd = 'ffmpeg -hide_banner -loglevel panic -ss {} -t {} -i "{}" -y "{}"'.format(
            split_start,
            split_size,
            input,
            output_path
        )
        # print(cmd)
        check_call(shlex.split(cmd), universal_newlines=True)
        output.append(output_path)
    return output

In [ ]:
print(c_unique_glosses)

{'GLOSS': 0, 'ZELFDE': 1, 'REGEN': 2, 'MOOI': 3, 'ALLES': 4, 'ORGANISEREN': 5, 'OPRICHTEN': 6, 'ENZOVOORTS': 7, 'AL': 8, 'BEWAREN': 9, 'RESTAURANT': 10, 'RUIMTE': 11, 'WEG': 12, 'PT': 13, 'ERG': 14, 'HOPEN': 15, 'VREEMD': 16, 'KLOPT': 17, 'VORIGE': 18, 'PIJN': 19, 'DENKEN': 20, 'WANDELEN': 21, 'HEET': 22, 'MAAR': 23, 'DAG': 24, '2': 25, '2-ORD': 26, '3-ORD': 27, 'TOCH': 28, 'PO': 29, 'DRINKEN': 30, 'ETEN': 31, 'GELD': 32, 'VERLIEZEN': 33, 'MEE': 34, 'SLECHT': 35, 'WEER': 36, 'NIEUW': 37, 'MOVE': 38, 'ZON': 39, 'DOOR': 40, 'POPT': 41, 'OPGEVEN': 42, 'LICHAAM': 43, 'BRAND': 44, 'MOE': 45, 'SOLDAAT': 46, 'RUGZAK': 47, 'MENS': 48, 'OVERLEDEN': 49, '3': 50, 'NAAST': 51, '10': 52, 'OVERGEVEN': 53, 'REANIMEREN': 54, 'R': 55, 'KLAAR': 56, 'DAN': 57, 'BETER': 58, 'HERFST': 59, '4': 60, 'TOT': 61, 'JAAR': 62, 'VROEGER': 63, 'NOOIT': 64, 'KLAGEN': 65, '5': 66, 'GEBEUREN': 67, 'VOOR': 68, 'GELOVEN': 69, 'BLIJKEN': 70, 'WACHTEN': 71, 'LAAT-MAAR': 72, 'NU': 73, 'BOVEN': 74, '30': 75, 'ONGEVEER': 76,

In [20]:
## Here I split all the glossed videos by gloss and save as gloss chunks for CorpusNGT

datapath = "Data/CorpusNGT/gloss/"
glossvideopath = "Data/CorpusNGT/gloss_split/"
ext = 'mpg'

gloss_counter = dict([(i, 0) for i in c_unique_glosses]) # start counting the glosses so that we can give each split video a number so they don't overwrite eachother
error_count = 0

for filename in os.listdir(datapath):
    file, extension = filename.split('.')
    if extension == ext:
        gloss = pd.read_csv((datapath + file + ".csv"), header=None, names = ['signer', 'start seconds', 'start time', 'stop seconds', 'stop time', 'duration', 'value', 'gloss'])
        #gloss = gloss.loc[["Gloss LH", "Gloss RH"]]
        #gloss = gloss[~gloss.gloss.str.contains("(p-)")] ## removes unfinished annotations
        #gloss = gloss[~gloss.gloss.str.contains("(g-)")] ## removes unfinished annotations
        gloss['gloss'] = gloss['gloss'].apply(lambda x: cleanItem(x)) ## cleans up the glosses like we did with the statisitcs
        gloss['gloss'].replace('', np.nan, inplace=True)
        gloss.dropna(subset = ['gloss'], inplace = True)
        gloss = gloss.dropna(axis=0)
        gloss = gloss.iloc[1:]
        gloss = gloss.reset_index(drop=True)
        try:
            split_cut(datapath, glossvideopath, file, ext, gloss['start time'], gloss['stop time'], gloss['gloss'], gloss_counter)
        except Exception as err:
            error_count += 1
            print(Exception, err, error_count)

## We've now done it only for the videos with just glosses, we do the same for the videos with glosses and translations

#datapath = "drive/My Drive/Thesis/Data/ECHO/gloss_trans/"

## We keep counting where we've been counting from so no need to set gloss counter to 0

#for filename in os.listdir(datapath):
#    file, extension = filename.split('.')
#    if extension == ext:
#        print(file)
#        gloss = pd.read_csv((datapath + file + ".csv"), header=None, names = ['start seconds', 'start time', 'stop seconds', 'stop time', 'duration', 'value', 'gloss'])
#        gloss = gloss.dropna(axis=0)
#        gloss = gloss.loc[["Gloss RH", "Gloss LH"]]
#        gloss = gloss[~gloss.gloss.str.contains("(p-)")] ## removes unfinished annotations
#        gloss = gloss[~gloss.gloss.str.contains("(g-)")] ## removes unfinished annotations
#        gloss['gloss'] = gloss['gloss'].apply(lambda x: cleanItem(x)) ## cleans up the glosses like we did with the statisitcs

#        split_cut(datapath, glossvideopath, file, ext, gloss['start time'], gloss['stop time'], gloss['gloss'], gloss_counter)

Input #0, mpeg, from 'Data/CorpusNGT/gloss/CNGT0004_S003.mpg':
  Duration: 00:05:41.28, start: 0.187533, bitrate: 1730 kb/s
    Stream #0:0[0x1e0]: Video: mpeg1video, yuv420p(tv), 352x288 [SAR 178:163 DAR 1958:1467], 1500 kb/s, 25 fps, 25 tbr, 90k tbn, 25 tbc
    Stream #0:1[0x1c0]: Audio: mp2, 48000 Hz, mono, s16p, 192 kb/s
At least one output file must be specified

Input #0, mpeg, from 'Data/CorpusNGT/gloss/CNGT0004_S004.mpg':
  Duration: 00:05:41.28, start: 0.187533, bitrate: 1730 kb/s
    Stream #0:0[0x1e0]: Video: mpeg1video, yuv420p(tv), 352x288 [SAR 178:163 DAR 1958:1467], 1500 kb/s, 25 fps, 25 tbr, 90k tbn, 25 tbc
    Stream #0:1[0x1c0]: Audio: mp2, 48000 Hz, mono, s16p, 192 kb/s
At least one output file must be specified

Input #0, mpeg, from 'Data/CorpusNGT/gloss/CNGT0005_S003.mpg':
  Duration: 00:05:54.92, start: 0.187533, bitrate: 1730 kb/s
    Stream #0:0[0x1e0]: Video: mpeg1video, yuv420p(tv), 352x288 [SAR 178:163 DAR 1958:1467], 1500 kb/s, 25 fps, 25 tbr, 90k tbn, 25 tb

In [23]:
np.save('Data/CorpusNGT/gloss_counts.npy', gloss_counter)

In [ ]:
## Here I connect every translation to every gloss for the translated videos and save that as the new csv data file

datapath = "drive/My Drive/Thesis/Data/ECHO/gloss_trans/"
outputpath = "drive/My Drive/Thesis/Data/ECHO/"
ext = 'csv'

def check_interval(startgloss, stopgloss, startsent, stopsent):
    if startgloss >= startsent and stopgloss <=stopsent:
        return True
    else:
        return False

total_data_df = pd.DataFrame()

for filename in os.listdir(datapath):
    file, extension = filename.split('.')
    if extension == ext:
        print(file)
        gloss_location = os.path.join(datapath, filename)
        gloss = pd.read_csv(gloss_location, header=None, names = ['start seconds', 'start time', 'stop seconds', 'stop time', 'duration', 'value', 'item'])
        gloss = gloss.dropna(axis=0)
        gloss = gloss[~gloss.item.str.contains("(p-)")] ## removes unfinished annotations
        gloss = gloss[~gloss.item.str.contains("(g-)")] ## removes unfinished annotations
        gloss.rename(index={"Translation Dutch" : "Translation"},inplace=True)
        gloss.rename(index={"Translation Dutch " : "Translation"},inplace=True)
        gloss.rename(index={"Dutch Translation" : "Translation"},inplace=True)
        gloss.rename(index={"Dutch Translation " : "Translation"},inplace=True)
        sentences = gloss.loc[["Translation"]]                                  # copy dataframe with just translations to seperate df
        sentences['gloss'] = sentences.apply(lambda x: list(), axis=1)          # create another column with empty lists, we will add the glosses to this

        gloss = gloss.loc[["Gloss RH", "Gloss LH"]]
        gloss['item'] = gloss['item'].apply(lambda x: cleanItem(x))           # cleans up the glosses like we did with the statisitcs

        for i, sentence in sentences.iterrows():
            for j, glossrow in gloss.iterrows():
                if check_interval(glossrow['start time'], glossrow['stop time'], sentence['start time'], sentence['stop time']):
                    item = glossrow['item']
                    sentence['gloss'].append(item)


        total_data_df = pd.concat([total_data_df, sentences], ignore_index=False, sort=False)


In [ ]:
import string

total = total_data_df[total_data_df['gloss'].map(len) > 0]
total = total.rename(columns={"item": "sentence"})
total = total.reset_index(drop=True)

total['sentence'] = total['sentence'].apply(lambda x: (x.lower().translate(str.maketrans('', '', string.punctuation)) ))

total = total.drop(['start seconds',	'start time',	'stop seconds',	'stop time',	'duration',	'value'], axis=1)

In [ ]:
total.tail(10)

## After removing the entries with no glosses, we are left with just 483 sentences

,sentence,gloss
474,de kwaliteit bewijst het,"[BEWIJS, BEWIJS]"
475,dus laten we altijd zo verder gaan,"[LEKKER, RICHTING-OPGAAN]"
476,de aarde draait,[AARDE-DRAAIEN]
477,donkerte,"[VALLEN-VIA-HART, VALLEN-VIA-HART]"
478,innerlijke strijd,"[INNERLIJKE-STRIJD, INNERLIJKE-STRIJD]"
479,chaos,"[INNERLIJKE-CHAOS, INNERLIJKE-CHAOS]"
480,donkerte,"[VALLEN-VIA-HART, VALLEN-VIA-HART]"
481,yinyang,"[YIN-YANG, YIN-YANG]"
482,een bonzend hart,"[YIN-YANG-IN-HART-KLOPPEN, YIN-YANG-IN-HART-KL..."
483,de zon gaat langzaam onder,"[ZON-ONDERGAAN, AARDE]"


In [ ]:
# Save the dataframe to csv
total.to_csv('drive/My Drive/Thesis/Data/ECHO/sentences_with_gloss.csv')



---



Turn each video into frames


In [24]:
def get_frames(filename, n_frames= 1):
    frames = []
    v_cap = cv.VideoCapture(filename)
    v_len = int(v_cap.get(cv.CAP_PROP_FRAME_COUNT))
    frame_list= np.linspace(0, v_len-1, n_frames+1, dtype=np.int16)

    for fn in range(v_len):
        success, frame = v_cap.read()
        if success is False:
            continue
        if (fn in frame_list):
            frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
            frames.append(frame)
    v_cap.release()
    return frames, v_len

def store_frames(frames, path2store):
    for ii, frame in enumerate(frames):
        frame = cv.cvtColor(frame, cv.COLOR_RGB2BGR)
        path2img = os.path.join(path2store, "frame"+str(ii)+".jpg")
        cv.imwrite(path2img, frame)

In [25]:
from tqdm import tqdm

path2data = "Data/CorpusNGT/gloss_split/"

unique_glosses = np.load('Data/CorpusNGT/gloss_labels.npy',allow_pickle='TRUE').item()
#gloss_counter = np.load('Data/CorpusNGT/gloss_counts.npy',allow_pickle='TRUE').item()
print(unique_glosses)

for item in unique_glosses:
  sub_folder = item
  path2gloss = os.path.join(path2data, sub_folder)

  listOfCategories = os.listdir(path2gloss)
  listOfCategories, len(listOfCategories)

  extension = ".mpg"
  n_frames = 8
  for root, dirs, files in tqdm(os.walk(path2gloss, topdown=False)):
      for name in files:
          if extension not in name:
              continue
          path2vid = os.path.join(root, name)
          frames, vlen = get_frames(path2vid, n_frames= n_frames)
          path2store = path2vid.replace(extension, "")
          #print(path2store)
          os.makedirs(path2store, exist_ok= True)
          store_frames(frames, path2store)
      print("-"*50)

{'GLOSS': 0, 'LANG': 1, 'PO': 2, 'ZONDAG': 3, 'FIETSEN': 4, 'LOPEN': 5, 'WATER': 6, 'SPRINGEN': 7, 'STAAN': 8, 'BEEN': 9, 'GEK': 10, 'HOND': 11, 'DUWEN': 12, 'MOVE': 13, 'MEE': 14, 'GOOIEN': 15, 'KLAAR': 16, 'KLEREN': 17, 'VOCHTIG': 18, 'LACHEN': 19, 'UITKLEDEN': 20, 'VAN': 21, 'BROEK': 22, 'AANKLEDEN': 23, 'AL': 24, 'WACHTEN': 25, 'DROOG': 26, 'WEG': 27, 'BROOD': 28, 'JAM': 29, 'ONDERWIJS': 30, 'INTERNAAT': 31, 'ONGESTELD': 32, 'VERSCHILLENDE': 33, 'DUS': 34, 'ZENUWACHTIG': 35, 'OPGELUCHT': 36, 'BETER': 37, 'VERSLAAN': 38, 'GOED': 39, 'PANTOFFELS': 40, 'BE': 41, 'UUR': 42, 'BINNEN': 43, 'MOETEN': 44, 'LAATSTE': 45, 'PT': 46, 'MOOI': 47, 'HANDEN-WRIJVEN': 48, 'SCHRIKKEN': 49, 'SCHREEUWEN': 50, 'VLUCHTEN': 51, 'KAPOT': 52, 'HOUDEN-VAN': 53, 'SCHOOL': 54, 'ZATERDAG': 55, 'KAART': 56, 'BEZOEK': 57, 'TREIN': 58, 'BRIEF': 59, 'KUNNEN': 60, 'WAAR': 61, 'OPEN': 62, 'GEZELLIG': 63, 'PRATEN': 64, 'VERHAAL': 65, 'PRECIES': 66, 'KERK': 67, 'WIJ': 68, 'GAAN': 69, 'S': 70, 'MOEDER': 71, 'RUIMTE': 7

1it [00:00, 993.91it/s]


--------------------------------------------------


1it [00:02,  2.44s/it]


--------------------------------------------------


...

1it [00:00, 16.39it/s]


--------------------------------------------------


1it [00:00, 16.80it/s]


--------------------------------------------------


NotADirectoryError: [WinError 267] The directory name is invalid: 'Data/CorpusNGT/gloss_split/LIFT:ASL'